# Building Fast Queries on a CSV

## Description:

The aim of this project is to build a way to answer a few different business questions about the inventory in an online laptop store.

## Reading the Inventory

In [1]:
# import the csv module
import csv 

# read the laptops.csv
with open('laptops.csv') as f:
    read_file = csv.reader(f)
    rows = list(read_file)
    header = rows[0]
    rows = rows[1:]
    
#print the value of header and first five rows in rows
print(header)
for i in range(5):
    print(rows[i])
    

['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']
['6571244', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 2.3GHz', '8GB', '128GB SSD', 'Intel Iris Plus Graphics 640', 'macOS', '1.37kg', '1339']
['7287764', 'Apple', 'Macbook Air', 'Ultrabook', '13.3', '1440x900', 'Intel Core i5 1.8GHz', '8GB', '128GB Flash Storage', 'Intel HD Graphics 6000', 'macOS', '1.34kg', '898']
['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', '575']
['9722156', 'Apple', 'MacBook Pro', 'Ultrabook', '15.4', 'IPS Panel Retina Display 2880x1800', 'Intel Core i7 2.7GHz', '16GB', '512GB SSD', 'AMD Radeon Pro 455', 'macOS', '1.83kg', '2537']
['8550527', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 3.1GHz', '8GB', '256GB SSD',

## Inventory Class

The goal of this guided project is to create a class that represents our inventory. The methods in that class will implement the queries that we want to answer about our inventory. We will also preprocess that data to make those queries run faster.

Here are some queries that we will want to answer:

* Given a laptop id, find the corresponding data.

* Given an amount of money, find whether there are two laptops whose total price is that given amount.

* Identify all laptops whose price falls within a given budget.


In [2]:
# Create a class named Inventory()

class Inventory():
    def __init__(self, csv_filename):
        with open(csv_filename) as f:  
            read_file = csv.reader(f)
            rows = list(read_file)
            self.header = rows[0]
            self.rows = rows[1:]
            for row in self.rows:
                row[-1] = int(row[-1])


# Test your class by creating an instance of Inventory using 'laptops.csv' as argument.            
inventory = Inventory('laptops.csv')

# Print the headers by printing the value of the header property.
print(inventory.header)

# Using the len() function, print the number of rows. You should have 1303 rows.
print(len(inventory.rows))
        
        

['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']
1303


## Finding a Laptop From the Id

The first thing that we will implement is a way to look up a laptop from a given identifier. In this way, when a customer comes to our store with a purchase slip, we can quickly identify the laptop to which it corresponds.

For this, we will write a function named **get_laptop_from_id()**. This function will take as argument the identifier of the laptop and return the full row of the laptop with that id.


In [3]:
# modifying the inventory class

class Inventory():
    def __init__(self, csv_filename):
        with open(csv_filename) as f:  
            read_file = csv.reader(f)
            rows = list(read_file)
            self.header = rows[0]
            self.rows = rows[1:]
            for row in self.rows:
                row[-1] = int(row[-1])
    
    def get_laptop_from_id(self, laptop_id):   
        for row in self.rows:                  
            if row[0] == laptop_id:
                return row
        return None          

In [4]:
# let us test the class by creating an instance of Inventory using 'laptops.csv' as argument.

inventory = Inventory('laptops.csv')           # step 4
print(inventory.get_laptop_from_id('3362737')) # step 5
print(inventory.get_laptop_from_id('3362736')) # step 6

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]
None


## Improving Id Lookups

The previous algorithm requires us to look at every single row to find the one that we are looking for (or decide that such a row does not exist). This algorithm has time complexity O(R) where R is the number of rows.

We can solve this problem more efficiently by preprocessing the data. Using a set, we can check in constant time whether a given identifier exists. However, we don't just want to know if it exists, we also want to retrieve the remaining row information. Therefore, we will use a dictionary instead of a set. Dictionaries have the same fast lookup properties that sets have, but allow us to associate values to the keys.

In our dataset, we only have about 1,300 laptops, so it might seem unnecessary to improve the performance of this query. However, you have to imagine that this code could be used in situations where the inventory contains millions of rows. Also, if we perform a lot of queries, even on a small dataset, the slow query performance will start to add up. It might eventually become the bottleneck of the application.

The idea is preprocess the data into a dictionary where the keys are the IDs and the values the rows. Then, we will use that dictionary in the get_laptop_from_id() method. We can do this by:

* Preprocess the data and create the dictionary in the __init__() method.

* Re-implement the get_laptop_from_id() method. We will do it as a new method so that we can compare the two.

In [5]:
## modifying the inventory class

class Inventory():                    
    
    def __init__(self, csv_filename):
        with open(csv_filename) as f: 
            reader = csv.reader(f)
            rows = list(reader)
        self.header = rows[0]        
        self.rows = rows[1:]
        for row in self.rows:              
            row[-1] = int(row[-1])
        self.id_to_row = {}                         
        for row in self.rows:                       
            self.id_to_row[row[0]] = row 
    
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:                 
            if row[0] == laptop_id:
                return row
        return None   
    
    # Create a new method named get_laptop_from_id_fast() with arguments: 
    # self and laptop_id.
    
    def get_laptop_from_id_fast(self, laptop_id):   
        if laptop_id in self.id_to_row:             
            return self.id_to_row[laptop_id]
        return None

In [6]:
# Let us test the class by creating an instance of Inventory using 
# 'laptops.csv'

inventory_fast = Inventory('laptops.csv')
print(inventory_fast.get_laptop_from_id_fast('3362737'))
print(inventory_fast.get_laptop_from_id_fast('3362736'))

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]
None


## Comparing the Performance

The get_laptop_from_id() method has time complexity O(R) where R is the number of rows. In contrast, the new implementation is time complexity O(1). It does so by using more memory to store the self.id_to_row dictionary and using a bit more time creating an instance (because it needs to create the dictionary).

Let's experiment to compare the performance of the two methods. The idea is to generate random IDs using the random module. Then, use both methods to lookup these same IDs. We will use the time module to measure the execution time of each lookup and, for each method, add all times together.

In [7]:
import time                                                         
import random                                                       

ids = [str(random.randint(1000000, 9999999)) for _ in range(10000)] 

inventory = Inventory('laptops.csv')                                

total_time_no_dict = 0                                              
for identifier in ids:                                              
    start = time.time()                                             
    inventory.get_laptop_from_id(identifier)                        
    end = time.time()                                               
    total_time_no_dict += end - start                               
    
total_time_dict = 0                                                 
for identifier in ids:                                              
    start = time.time()                                             
    inventory_fast.get_laptop_from_id_fast(identifier)                   
    end = time.time()                                               
    total_time_dict += end - start                                  
    
print(total_time_no_dict)                                           
print(total_time_dict)
    

1.6032984256744385
0.0053577423095703125


## Analysis

We got:

1.7247488498687744

0.0056951045989990234

There is a significant improvement in performance. If we divide 1.7247 by 0.0057 we see that the new method is about 302 times faster for this input size.

## Two Laptop Promotion

Sometimes, the store offers a promotion where they give a gift card. A customer can use the gift to buy up to two laptops. To avoid having to keep track of what was already spent, the gift card has a single time usage. This means that, even if there is leftover money, it cannot be used anymore.

We don't want to make a customer feel cheated, so whenever they are issued a gift card, we want to make sure that there is at least one way to spend it in full. In other words, before issuing a gift card for D dollars, we want to make sure that either there is a laptop that costs exactly D dollars or two laptops whose costs add up to precisely D dollars.

We will write a function that, given a dollar amount, checks whether it is possible to spend precisely that amount by purchasing up to two laptops.

In [8]:

class Inventory():                    
    
    def __init__(self, csv_filename):
        with open(csv_filename) as f: 
            reader = csv.reader(f)
            rows = list(reader)
        self.header = rows[0]        
        self.rows = rows[1:]
        for row in self.rows:              
            row[-1] = int(row[-1])
        self.id_to_row = {}                         
        for row in self.rows:                       
            self.id_to_row[row[0]] = row 
    
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:                 
            if row[0] == laptop_id:
                return row
        return None   
    
    def get_laptop_from_id_fast(self, laptop_id):   
        if laptop_id in self.id_to_row:             
            return self.id_to_row[laptop_id]
        return None
    
# Create a method named check_promotion_dollars() that takes two arguments: 
# self and dollars.

    
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            if row[-1] == dollars:
                return True
        for row1 in self.rows:
            for row2 in self.rows:
                if row1[-1] + row2[-1] == dollars:
                    return True
        return False
                

## Test the class

In [9]:
inventory = Inventory('laptops.csv')               
print(inventory.check_promotion_dollars(1000))     
print(inventory.check_promotion_dollars(442))      

True
False


## Optimizing Laptop Promotion

We've seen how we can preprocess data to answer the kind of queries that we used in the check_promotion_dollars(). Let's implement this to make our code run faster.

Since we only care about whether or not there is a solution, we can store all laptops prices in a set when we initialize the inventory. Then we can check in constant time whether there is a laptop with a given price.

To check if there is a pair of laptops can be done in the same way as we did previously.

In [10]:
class Inventory():                    
# At the end of the __init__() method, assign an empty set to self.prices     
# Loop over all rows and add the price contained in that row to self.prices    
    
    def __init__(self, csv_filename):
        with open(csv_filename) as f: 
            reader = csv.reader(f)
            rows = list(reader)
        self.header = rows[0]        
        self.rows = rows[1:]
        for row in self.rows:              
            row[-1] = int(row[-1])
        self.id_to_row = {}                        
        for row in self.rows:                       
            self.id_to_row[row[0]] = row
        self.prices = set()                          
        for row in self.rows:                        
            self.prices.add(row[-1])
    
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:                 
            if row[0] == laptop_id:
                return row
        return None   
    
    def get_laptop_from_id_fast(self, laptop_id):  
        if laptop_id in self.id_to_row:           
            return self.id_to_row[laptop_id]
        return None

    def check_promotion_dollars(self, dollars):    
        for row in self.rows:                   
            if row[-1] == dollars:
                return True
        for row1 in self.rows:                  
            for row2 in self.rows:
                if row1[-1] + row2[-1] == dollars:
                    return True
        return False                        
    
    # Create a method named check_promotion_dollars_fast() that takes two arguments: 
    # self and dollars 
    def check_promotion_dollars_fast(self, dollars): 
        if dollars in self.prices:                   
            return True
        for price in self.prices:                    
            if dollars - price in self.prices:
                return True
        return False           
                

In [11]:
inventory = Inventory('laptops.csv')                 
print(inventory.check_promotion_dollars_fast(1000))  
print(inventory.check_promotion_dollars_fast(442))   

True
False


## Comparing Promotion Functions

Let's compare the performance of the last two functions that we wrote.

In [12]:
# Generate a list named prices with 100 random values between 100 and 5,000.
prices = [random.randint(100, 5000) for _ in range(100)] 

inventory = Inventory('laptops.csv')                     

total_time_no_set = 0                                    
for price in prices:                                     
    start = time.time()                                  
    inventory.check_promotion_dollars(price)             
    end = time.time()                                    
    total_time_no_set += end - start                     
    
total_time_set = 0                                       
for price in prices:                                     
    start = time.time()                                  
    inventory.check_promotion_dollars_fast(price)        
    end = time.time()                                    
    total_time_set += end - start                        
    
print(total_time_no_set)                                 
print(total_time_set)

3.110440492630005
0.0011408329010009766


## Analysis

From the results, we can see there is a significant increase in performance. If we divide 3.230 by 0.001, we see that the new method is 3,230 times faster for this input size. 

## Finding Laptops Within a Budget

We have seen how we use binary search to find an element in a sorted list quickly. In this project, we are going to leverage and extend that algorithm to help a customer find all laptops that fall within their budget.

More formally, we want to write a method that efficiently answers the query: Given a budget of D dollars, find all laptops whose price it at most D.

In [13]:
def row_price(row):
    return row[-1]

class Inventory():                    
    
    # At the end of the __init__() method, use the sorted() function to sort 
    # the rows by price and assign the result to self.rows_by_price
    
    def __init__(self, csv_filename):
        with open(csv_filename) as f: 
            reader = csv.reader(f)
            rows = list(reader)
        self.header = rows[0]        
        self.rows = rows[1:]
        for row in self.rows:              
            row[-1] = int(row[-1])
        self.id_to_row = {}                        
        for row in self.rows:                       
            self.id_to_row[row[0]] = row
        self.prices = set()                          
        for row in self.rows:                        
            self.prices.add(row[-1])
        self.rows_by_price = sorted(self.rows, key=row_price) 
    
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:                 
            if row[0] == laptop_id:
                return row
        return None   
    
    def get_laptop_from_id_fast(self, laptop_id):  
        if laptop_id in self.id_to_row:           
            return self.id_to_row[laptop_id]
        return None

    def check_promotion_dollars(self, dollars):    
        for row in self.rows:                   
            if row[-1] == dollars:
                return True
        for row1 in self.rows:                  
            for row2 in self.rows:
                if row1[-1] + row2[-1] == dollars:
                    return True
        return False                        
    
    def check_promotion_dollars_fast(self, dollars):
        if dollars in self.prices:                   
            return True
        for price in self.prices:                    
            if dollars - price in self.prices:
                return True
        return False                                
    
    def find_laptop_with_price(self, target_price):
        range_start = 0                                   
        range_end = len(self.rows_by_price) - 1                       
        while range_start < range_end:
            range_middle = (range_end + range_start) // 2  
            value = self.rows_by_price[range_middle][-1]
            if value == target_price:                            
                return range_middle                        
            elif value < target_price:                           
                range_start = range_middle + 1             
            else:                                          
                range_end = range_middle - 1 
        if self.rows_by_price[range_start][-1] != target_price:                  
            return -1                                      
        return range_start
    
    # Implement a method named find_first_laptop_more_expensive().
    # This method should take two arguments: self and price
    
    def find_first_laptop_more_expensive(self, target_price): 
        range_start = 0                                   
        range_end = len(self.rows_by_price) - 1                   
        while range_start < range_end:
            range_middle = (range_end + range_start) // 2  
            price = self.rows_by_price[range_middle][-1]
            if price > target_price:
                range_end = range_middle
            else:
                range_start = range_middle + 1
        if self.rows_by_price[range_start][-1] <= target_price:                  
            return -1                                   
        return range_start

inventory = Inventory('laptops.csv')                                 
print(inventory.find_first_laptop_more_expensive(1000))  
print(inventory.find_first_laptop_more_expensive(10000))  

683
-1


## Conclusion

In this guided project, we only explored three possible queries that we might want to do over the data. In general, we often have a lot of different datasets to process and queries to answer. Designing such a class for every type of data in a business and implementing specific query methods takes a lot of time.